In [1]:
import pandas as pd
import numpy as np


In [18]:
df032022 = pd.read_csv(r"c:\Users\Tim\OneDrive\Desktop\thesis\filtered_csv\03-2022-filter")

In [3]:
df042022 = pd.read_csv(r"c:\Users\Tim\OneDrive\Desktop\thesis\filtered_csv\04-2022-filter")

In [4]:
df052022 = pd.read_csv(r"c:\Users\Tim\OneDrive\Desktop\thesis\filtered_csv\05-2022-filter")

In [5]:
df062022 = pd.read_csv(r"c:\Users\Tim\OneDrive\Desktop\thesis\filtered_csv\06-2022-filter")

In [6]:
df072022 = pd.read_csv(r"c:\Users\Tim\OneDrive\Desktop\thesis\filtered_csv\07-2022-filter")

In [7]:
df082022 = pd.read_csv(r"c:\Users\Tim\OneDrive\Desktop\thesis\filtered_csv\08-2022-filter")

In [8]:
df092022 = pd.read_csv(r"c:\Users\Tim\OneDrive\Desktop\thesis\filtered_csv\09-2022-filter")

In [9]:
df102022 = pd.read_csv(r"c:\Users\Tim\OneDrive\Desktop\thesis\filtered_csv\10-2022-filter")

In [10]:
df112022 = pd.read_csv(r"c:\Users\Tim\OneDrive\Desktop\thesis\filtered_csv\11-2022-filter")

In [11]:
df122022 = pd.read_csv(r"c:\Users\Tim\OneDrive\Desktop\thesis\filtered_csv\12-2022-filter")

In [12]:
df012023 = pd.read_csv(r"c:\Users\Tim\OneDrive\Desktop\thesis\filtered_csv\01-2023-filter")

In [13]:
df022023 = pd.read_csv(r"c:\Users\Tim\OneDrive\Desktop\thesis\filtered_csv\02-2023-filter")

In [14]:
df032023 = pd.read_csv(r"c:\Users\Tim\OneDrive\Desktop\thesis\filtered_csv\03-2023-filter")

In [15]:
df_3mp = pd.concat([df032022, df042022, df052022, df062022], ignore_index=True)


In [61]:
df3_mp_cumulative = df_3mp.groupby('user_login', as_index=False).sum()


In [17]:
import pandas as pd
from ast import literal_eval
from datetime import datetime

# Assuming df_3mp is your DataFrame

# Convert string representations of lists back to lists if they're not already
df_3mp['event_dates'] = df_3mp['event_dates'].apply(literal_eval)
df_3mp['event_types'] = df_3mp['event_types'].apply(literal_eval)

# Define start and end dates for the filtering
start_date = datetime(2022, 3, 22)
end_date = datetime(2022, 6, 22)

# Function to check if any date in the list falls within the desired range
def is_within_date_range(date_list):
    return any(start_date <= datetime.strptime(date, '%Y-%m-%d') <= end_date for date in date_list)

# Filter the DataFrame
filtered_df = df_3mp[df_3mp['event_dates'].apply(is_within_date_range)]

# Custom aggregation function for concatenating lists
def concat_lists(series):
    aggregated_list = []
    for item in series:
        aggregated_list.extend(item)
    return aggregated_list

# Define aggregation functions for each column
aggregation_functions = {
    'event_dates': concat_lists,  # Concatenate dates
    'event_types': concat_lists,  # Concatenate types
    # Sum these columns; add or remove as necessary based on your DataFrame
    'event_counts': 'sum',
    'PullRequestEvent': 'sum',
    'PushEvent': 'sum',
    'IssuesEvent': 'sum',
    'CreateEvent': 'sum',
    'ForkEvent': 'sum'
}

# Apply aggregation on the filtered DataFrame
df3_mp_cumulative = filtered_df.groupby('user_login', as_index=False).agg(aggregation_functions)


KeyboardInterrupt: 

In [62]:
df3_mp_cumulative.head()

,user_login,event_counts,PullRequestEvent,PushEvent,IssuesEvent,CreateEvent,ForkEvent
0,0-melkonyanma-0,37,7,23,0,4,3
1,0-tae,51,0,47,0,4,0
2,0-v-0,78,2,53,0,20,3
3,0-vortex,548,168,295,40,41,4
4,0-wiz-0,163,8,120,17,9,9


In [63]:
df3_mp_cumulative.shape

(175585, 7)

In [33]:
df_3mp.head()

,user_login,event_dates,event_types,event_counts,PullRequestEvent,PushEvent,IssuesEvent,CreateEvent,ForkEvent
0,0-melkonyanma-0,"['2022-03-27', '2022-03-13', '2022-03-21', '20...","['PullRequestEvent', 'PushEvent', 'PushEvent',...",11,4,7,0,0,0
1,0-tae,"['2022-03-05', '2022-03-05', '2022-03-05', '20...","['PushEvent', 'PushEvent', 'PushEvent', 'PushE...",33,0,29,0,4,0
2,0-v-0,"['2022-03-19', '2022-03-24', '2022-03-04', '20...","['PushEvent', 'CreateEvent', 'PushEvent', 'Pus...",22,2,13,0,5,2
3,0-vortex,"['2022-03-12', '2022-03-11', '2022-03-06', '20...","['PushEvent', 'PullRequestEvent', 'PullRequest...",253,63,141,27,21,1
4,0-wiz-0,"['2022-03-26', '2022-03-08', '2022-03-26', '20...","['PushEvent', 'IssuesEvent', 'PushEvent', 'Pus...",49,2,41,4,0,2


In [64]:
import pandas as pd
from ast import literal_eval
from datetime import datetime

def process_events(df):
    # Convert string representations of lists to actual lists
    df['event_dates'] = df['event_dates'].apply(literal_eval)
    df['event_types'] = df['event_types'].apply(literal_eval)
    
    for index, row in df.iterrows():
        new_dates = []
        new_types = []
        # Reset the total event count and specific event type counts for accuracy
        row['event_counts'] = 0
        for event_type in ['PullRequestEvent', 'PushEvent', 'IssuesEvent', 'CreateEvent', 'ForkEvent']:
            if event_type in df.columns:
                row[event_type] = 0

        for date, event_type in zip(row['event_dates'], row['event_types']):
            # Check if the date is within the range 2022-03-22 to 2022-06-22
            if datetime.strptime(date, '%Y-%m-%d') >= datetime(2022, 3, 22) and datetime.strptime(date, '%Y-%m-%d') <= datetime(2022, 6, 22):
                new_dates.append(date)
                new_types.append(event_type)
                # Increment the event count and specific event type count
                row['event_counts'] += 1
                if event_type in df.columns:
                    row[event_type] += 1
        
        # Update the DataFrame with the modified lists and counts
        df.at[index, 'event_dates'] = str(new_dates)
        df.at[index, 'event_types'] = str(new_types)
    
    return df

In [65]:
df_3mp_processed = process_events(df3_mp_cumulative)

KeyboardInterrupt: 

In [34]:
import pandas as pd
from ast import literal_eval
from datetime import datetime
 #Function to process the DataFrame
def process_events(df):
    # Convert string representations of lists to actual lists
    df['event_dates'] = df['event_dates'].apply(literal_eval)
    df['event_types'] = df['event_types'].apply(literal_eval)
    
    # Iterate through each row and process
    for index, row in df.iterrows():
        new_dates = []
        new_types = []
        for date, event_type in zip(row['event_dates'], row['event_types']):
            # Check if the date is before 2022-03-22
            if datetime.strptime(date, '%Y-%m-%d') >= datetime(2022, 3, 22):
                new_dates.append(date)
                new_types.append(event_type)
            else:
                # Decrement the event count and specific event type count
                row['event_counts'] -= 1
                if event_type in df.columns:
                    row[event_type] -= 1
        
        # Update the DataFrame with the modified lists
        df.at[index, 'event_dates'] = str(new_dates)
        df.at[index, 'event_types'] = str(new_types)
    
    return df

In [36]:
df_3mp_processed = process_events(df_3mp)

In [38]:
df_3mp_processed.head(20)

,user_login,event_dates,event_types,event_counts,PullRequestEvent,PushEvent,IssuesEvent,CreateEvent,ForkEvent
0,0-melkonyanma-0,"['2022-03-27', '2022-03-27']","['PullRequestEvent', 'PushEvent']",11,4,7,0,0,0
1,0-tae,"['2022-03-25', '2022-03-30', '2022-03-29', '20...","['PushEvent', 'PushEvent', 'PushEvent', 'PushE...",33,0,29,0,4,0
2,0-v-0,"['2022-03-24', '2022-03-25', '2022-03-23', '20...","['CreateEvent', 'PushEvent', 'PushEvent', 'Cre...",22,2,13,0,5,2
3,0-vortex,"['2022-03-25', '2022-03-25', '2022-03-28', '20...","['PushEvent', 'PullRequestEvent', 'PushEvent',...",253,63,141,27,21,1
4,0-wiz-0,"['2022-03-26', '2022-03-26', '2022-03-28', '20...","['PushEvent', 'PushEvent', 'PullRequestEvent',...",49,2,41,4,0,2
5,00-Evan,"['2022-03-23', '2022-03-23', '2022-03-25', '20...","['PushEvent', 'IssuesEvent', 'IssuesEvent', 'I...",38,0,6,27,5,0
6,0000000100000000,"['2022-03-22', '2022-03-22', '2022-03-25', '20...","['ForkEvent', 'ForkEvent', 'ForkEvent', 'ForkE...",39,0,0,0,0,39
7,0000Blaze,[],[],45,0,40,0,4,1
8,0000OOOO0000,"['2022-03-27', '2022-03-25', '2022-03-25', '20...","['PushEvent', 'PushEvent', 'PushEvent', 'PushE...",39,0,39,0,0,0
9,000744210,[],[],1,0,1,0,0,0


In [39]:
# Assuming 'user_login' is the column name and 'irahorecka' is the user of interest
user_row = df_3mp_processed[df_3mp_processed['user_login'] == '000paradox000']

In [40]:
event_dates_list = user_row.iloc[0]['event_dates']

# Print or process the event_dates_list
print(event_dates_list)

['2022-03-28', '2022-03-22', '2022-03-28', '2022-03-28', '2022-03-22', '2022-03-22', '2022-03-28', '2022-03-28', '2022-03-28', '2022-03-28', '2022-03-28', '2022-03-28', '2022-03-28', '2022-03-28', '2022-03-28', '2022-03-28', '2022-03-28', '2022-03-23', '2022-03-28', '2022-03-28', '2022-03-29', '2022-03-29', '2022-03-28', '2022-03-28', '2022-03-28', '2022-03-22', '2022-03-28', '2022-03-28', '2022-03-28', '2022-03-28', '2022-03-25', '2022-03-23', '2022-03-28', '2022-03-29', '2022-03-28', '2022-03-24', '2022-03-28', '2022-03-28', '2022-03-24', '2022-03-28', '2022-03-28', '2022-03-22', '2022-03-22', '2022-03-28', '2022-03-25', '2022-03-25', '2022-03-28', '2022-03-28', '2022-03-28', '2022-03-29', '2022-03-28', '2022-03-23', '2022-03-28', '2022-03-25']


In [51]:
df_3ma = pd.concat([df062022, df072022, df082022, df092022], ignore_index=True)


In [54]:
df072022.head()

,user_login,event_dates,event_types,event_counts,PullRequestEvent,PushEvent,IssuesEvent,CreateEvent,ForkEvent
0,0-melkonyanma-0,"['2022-07-28', '2022-07-28', '2022-07-29', '20...","['PushEvent', 'PushEvent', 'PushEvent', 'PushE...",33,0,31,0,2,0
1,0-tae,"['2022-07-05', '2022-07-26', '2022-07-05', '20...","['PushEvent', 'PushEvent', 'PushEvent', 'PushE...",16,0,16,0,0,0
2,0-v-0,"['2022-07-16', '2022-07-17', '2022-07-26', '20...","['PushEvent', 'PullRequestEvent', 'PushEvent',...",34,3,29,0,1,1
3,0-vortex,"['2022-07-31', '2022-07-31', '2022-07-30', '20...","['PushEvent', 'PushEvent', 'IssuesEvent', 'Pus...",188,29,108,46,5,0
4,0-wiz-0,"['2022-07-19', '2022-07-29', '2022-07-19', '20...","['PullRequestEvent', 'IssuesEvent', 'PushEvent...",24,4,8,9,3,0


In [52]:
df_3ma.head(20)

,user_login,event_dates,event_types,event_counts,PullRequestEvent,PushEvent,IssuesEvent,CreateEvent,ForkEvent
0,0-melkonyanma-0,['2022-06-08'],['PushEvent'],1,0,1,0,0,0
1,0-tae,"['2022-06-27', '2022-06-22', '2022-06-29']","['PushEvent', 'PushEvent', 'PushEvent']",3,0,3,0,0,0
2,0-v-0,"['2022-06-23', '2022-06-16', '2022-06-29', '20...","['PushEvent', 'PushEvent', 'CreateEvent', 'Pus...",22,0,18,0,3,1
3,0-vortex,"['2022-06-06', '2022-06-13', '2022-06-29', '20...","['PullRequestEvent', 'PullRequestEvent', 'Push...",66,23,36,4,3,0
4,0-wiz-0,"['2022-06-28', '2022-06-13', '2022-06-13', '20...","['IssuesEvent', 'PushEvent', 'PushEvent', 'For...",22,3,10,5,1,3
5,00-Evan,"['2022-06-13', '2022-06-24', '2022-06-05', '20...","['PushEvent', 'IssuesEvent', 'IssuesEvent', 'I...",19,1,5,11,1,1
6,0000000100000000,"['2022-06-08', '2022-06-14', '2022-06-26', '20...","['ForkEvent', 'ForkEvent', 'ForkEvent', 'ForkE...",25,0,0,0,0,25
7,0000Blaze,"['2022-06-01', '2022-06-12', '2022-06-07', '20...","['PushEvent', 'PushEvent', 'PushEvent', 'Creat...",31,0,25,0,4,2
8,0000OOOO0000,"['2022-06-23', '2022-06-07', '2022-06-09', '20...","['PushEvent', 'PushEvent', 'PushEvent', 'PushE...",136,0,136,0,0,0
9,000744210,"['2022-06-23', '2022-06-09', '2022-06-26', '20...","['PushEvent', 'PushEvent', 'PushEvent', 'PushE...",34,0,34,0,0,0


In [45]:
columns_to_drop = [
    'PullRequestEvent', 'PushEvent', 'IssuesEvent', 'CreateEvent', 'ForkEvent'
]

# Assuming your DataFrame is named df
df_3ma = df_3ma.drop(columns=columns_to_drop)

In [59]:
df_3ma.head()

,user_login,event_dates,event_types,event_counts,PullRequestEvent,PushEvent,IssuesEvent,CreateEvent,ForkEvent
0,0-melkonyanma-0,['2022-06-08'],['PushEvent'],1,0,1,0,0,0
1,0-tae,"['2022-06-27', '2022-06-22', '2022-06-29']","['PushEvent', 'PushEvent', 'PushEvent']",3,0,3,0,0,0
2,0-v-0,"['2022-06-23', '2022-06-16', '2022-06-29', '20...","['PushEvent', 'PushEvent', 'CreateEvent', 'Pus...",22,0,18,0,3,1
3,0-vortex,"['2022-06-06', '2022-06-13', '2022-06-29', '20...","['PullRequestEvent', 'PullRequestEvent', 'Push...",66,23,36,4,3,0
4,0-wiz-0,"['2022-06-28', '2022-06-13', '2022-06-13', '20...","['IssuesEvent', 'PushEvent', 'PushEvent', 'For...",22,3,10,5,1,3


In [49]:
df3_ma_cumulative = df_3ma.groupby('user_login', as_index=False).sum()

In [50]:
df3_ma_cumulative.head()

,user_login,event_counts
0,0-melkonyanma-0,84
1,0-tae,70
2,0-v-0,104
3,0-vortex,552
4,0-wiz-0,97
